# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-12-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

print(file_path_list)

# for every filepath in the file path list 
for f in file_path_list:
    print(f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/work

In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project3 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project3')
except Exception as e:
    print(e)

## Three Queries to ask the following questions of the data





### Query 1: 
#### Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [42]:
# Create table
query_create = "CREATE TABLE IF NOT EXISTS song_library "
query_create = query_create + "(sessionId int, itemInSession int, artist_name text, song text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)

                    

##### The data was modelled with only the columns required for output. The partition was based on sessionId with a clustering column of itemInSession (both forming the Primary Key). Either itemInSession or sessionId could have worked well as a partition key. However, combining them together creates a unique key to query the data. 

In [43]:
# Read the data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements
        query_insert = "INSERT INTO song_library (sessionId, itemInSession, artist_name, song, song_length)"
        query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [44]:
# SELECT statement to verify the data was entered into the table
query_select = "select artist_name, song, song_length \
            from song_library \
            where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)

    
# use pretty table to display data in tabular form and include headings
table = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    table.add_row([row.artist_name, row.song, row.song_length])

print("The artist, song and song length for the session Id of 338 and itemInSession of 4")
print(table)


The artist, song and song length for the session Id of 338 and itemInSession of 4
+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### Query 2: 
#### Get only the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### The data was modelled with only the columns required for output. The partition was based on userId with a clustering column of itemInSession (both forming the Primary Key). Either itemInSession or userId could have worked well as a partition key. Combining them together creates a unique key to query the data.

In [45]:
# Create the table
query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(userId int, itemInSession int, sessionId int, artist_name text, song text, firstname text, lastname text, PRIMARY KEY (userId, itemInSession, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# Read the data from the CSV file
  
# file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements 
        query = "INSERT INTO user_library (userId, itemInSession, sessionId, artist_name, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[3]), int(line[8]), line[0], line[9], line[1], line[4]))

# SELECT statement
query = "select artist_name, song, firstname, lastname \
            from user_library \
            where userId=8 and itemInSession=1 \
            order by itemInSession ASC"
  
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use pretty table to display data in tabular form and include headings
table = PrettyTable(['Artist', 'Song', 'First Name','Last Name'])
try:
    for row in rows:
        table.add_row([row.artist_name, row.song, row.firstname, row.lastname])
except Exception as e:
    print(e)

print("The Artist & Songs the user with Id 10 and session Id 1 has listened to'")
print(table)

The Artist & Songs the user with Id 10 and session Id 1 has listened to'
+-------------------------------+------------------------+------------+-----------+
|             Artist            |          Song          | First Name | Last Name |
+-------------------------------+------------------------+------------+-----------+
|            Des'ree            |      You Gotta Be      |   Kaylee   |  Summers  |
|         Justin Bieber         |    Never Let You Go    |   Kaylee   |  Summers  |
|       ISRAEL & NEW BREED      |     Awesome Medley     |   Kaylee   |  Summers  |
|         Be Bop Deluxe         |       Axe Victim       |   Kaylee   |  Summers  |
|            Battles            |      Snare Hangar      |   Kaylee   |  Summers  |
|           John Mayer          |     War Of My Life     |   Kaylee   |  Summers  |
| The Velvet Underground / Nico | All Tomorrow's Parties |   Kaylee   |  Summers  |
+-------------------------------+------------------------+------------+-----------+


### Query 3: 
####  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### The data was modelled with only the columns required for output. The partition was based on song with no clustering columns. As song is unique I felt confident it could be used as the Primary Key. In addition, as we are filtering just by song, I thought it was only necessary to partition by this variable. 

In [46]:
# Create statement
query = "CREATE TABLE IF NOT EXISTS history_library "
query = query + "(song text, userId int, firstname text, lastname text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Read the data from the CSV file
  
# file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
        query = "INSERT INTO history_library (song, userId, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# SELECT statement
query = "select firstname, lastname \
            from history_library \
            where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# try:
#     for row in rows:
#         print (row.firstname, row.lastname)
# except Exception as e:
#     print(e)

# use pretty table to display data in tabular form and include headings
table = PrettyTable(['First Name','Last Name'])
try:
    for row in rows:
        table.add_row([row.firstname, row.lastname])
except Exception as e:
    print(e)

print("The names of users who have listened to the song 'All Hands Against His Own'")
print(table)                    

The names of users who have listened to the song 'All Hands Against His Own'
+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [41]:
query_drop = "DROP TABLE IF EXISTS song_library "
try:
    session.execute(query_drop)
except Exception as e:
    print(e)
    
query_drop = "DROP TABLE IF EXISTS user_library "
try:
    session.execute(query_drop)
except Exception as e:
    print(e)

query_drop = "DROP TABLE IF EXISTS history_library "
try:
    session.execute(query_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()